In [39]:
from tools import *
import os
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

import torch.nn.functional as F
from torch_geometric.nn import SAGEConv
from tqdm import tqdm  

Préparation et chargement des graphes pour l'entraînement du GNN.

In [40]:
graph_json_dir = "graphes_JSON"  
graph_json_files = [graph_json_dir+"/"+f for f in os.listdir(graph_json_dir) if f.endswith('.json')]

all_graphs = []

for path in graph_json_files:
    node_features_df=extract_node_features_from_json_file(path)
    edges_df=extract_mapped_edges_from_json(path)
    target_df=extract_optimal_repartition_from_json(path)

    node_features_tensor, edge_index_tensor, y_target_tensor = prepare_data_for_GNN(node_features_df, edges_df, target_df)
    data = Data(x=node_features_tensor, edge_index=edge_index_tensor, y=y_target_tensor)

    all_graphs.append(data)

# graph_loader = DataLoader(graphs, batch_size=4, shuffle=True)

# for batch in graph_loader:
#     node_features_batch = batch.x
#     edge_index_batch = batch.edge_index
#     target_batch = batch.y

print("finished data preparting & loading")

finished data preparting & loading


In [41]:
sample = all_graphs[0]
print(f"Number of nodes: {sample.num_nodes}")
print(f"Node features: {sample.x.shape}")
print(f"Edges: {sample.edge_index.shape} (2, num_edges)")
print(f"Targets: {sample.y.shape} (CPU assignments)")

Number of nodes: 135
Node features: torch.Size([135, 8])
Edges: torch.Size([2, 147]) (2, num_edges)
Targets: torch.Size([135]) (CPU assignments)


separer les données en train validation & test

In [42]:
train_data = all_graphs[:80]
val_data = all_graphs[80:90]
test_data = all_graphs[90:]

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8)
test_loader = DataLoader(test_data, batch_size=8)

In [ ]:
print(f'train: {len(train_data)} validation: {len(val_data)} test: {len(test_data)}')

Model training